In [1]:
%pip install petpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
def getbreeds():
    import petpy
    pf = petpy.Petfinder(key='2p8Gsyz5kSnyKFNp4w7XqmNWD5UDHgPJ5Qkn9gLp9XHbkXOp68', secret='EEjObz7j3VQ0XhrOgITmyiZUK4nTg0LtJMcGzHdb')
    cat_breeds_df = pf.breeds('cat', return_df = True)
    return cat_breeds_df

In [141]:
import petpy
import numpy as np
import pickle
pf = petpy.Petfinder(key='2p8Gsyz5kSnyKFNp4w7XqmNWD5UDHgPJ5Qkn9gLp9XHbkXOp68', secret='EEjObz7j3VQ0XhrOgITmyiZUK4nTg0LtJMcGzHdb')
cats_df = pf.animals(animal_type='cat', status='adoptable', results_per_page=50, return_df = True)

In [142]:
cats_df.columns

Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description', 'organization_animal_id',
       'photos', 'videos', 'status', 'status_changed_at', 'published_at',
       'distance', 'breeds.primary', 'breeds.secondary', 'breeds.mixed',
       'breeds.unknown', 'colors.primary', 'colors.secondary',
       'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'primary_photo_cropped.small', 'primary_photo_cropped.medium',
       'primary_photo_cropped.large', 'primary_photo_cropped.full',
       'contact.email', 'contact.phone', 'contact.address.address1',
       'contact.address.address2', 'contact.address.city',
       'contact.address.state', 'contact.address.postcode',
       'contact.address.country', 'animal_id', 'animal_type',

In [143]:
from datetime import datetime
from datetime import date
from sklearn import preprocessing
import pandas as pd

cattopnames = ['Oliver',
 'Max',
 'Bella',
 'X',
 'Lily',
 'Oreo',
 'Daisy',
 'Charlie',
 'Lucy',
 'Sam',
 'Luna',
 'Kitty',
 'Molly',
 'Sophie',
 'Tiger',
 'George',
 'Jack',
 'Oscar',
 'Lilly',
 'Leo']

cattopbreeds = ['Domestic Shorthair Mix',
 'Domestic Medium Hair Mix',
 'Domestic Longhair Mix',
 'Siamese Mix',
 'Domestic Shorthair']

def getModelResults(df):
    ages = {'Baby': 0, 'Young': 1, 'Adult': 2, 'Senior': 3}
    modeldf = df.copy(deep=True)
    modeldf["attributes.spayed_neutered"] = modeldf["attributes.spayed_neutered"].fillna("unknown")
    modeldf['year'] =  date.today().year
    modeldf['month'] =  date.today().month
    modeldf['day'] =  date.today().weekday()
    modeldf['age'] = modeldf['age'].map(ages)
    modeldf['age'] = modeldf['age'].fillna(2)
    modeldf['SexuponOutcome_Female-False'] = (modeldf['gender'] == 'Female') & (modeldf['attributes.spayed_neutered'] == False)
    modeldf['SexuponOutcome_Male-False'] = (modeldf['gender'] == 'Male') & (modeldf['attributes.spayed_neutered'] == False)
    modeldf['SexuponOutcome_Female-True'] = (modeldf['gender'] == 'Female') & (modeldf['attributes.spayed_neutered'] == True)
    modeldf['SexuponOutcome_Male-True'] = (modeldf['gender'] == 'Male') & (modeldf['attributes.spayed_neutered'] == True)
    modeldf["colors.primary"] = modeldf["colors.primary"].fillna("unknown")
    modeldf['SexuponOutcome_Unknown'] = modeldf['gender'].apply(lambda x: 0)
    modeldf = modeldf[["breeds.primary", "SexuponOutcome_Female-False", "SexuponOutcome_Male-False", "SexuponOutcome_Female-True", "SexuponOutcome_Male-True", "SexuponOutcome_Unknown", "year", "month", "day", "age", "colors.primary", "name"]]
    le = preprocessing.LabelEncoder()
    modeldf['colors.primary'] = le.fit_transform(modeldf['colors.primary'])
    
    modeldf['name'] = modeldf['name'].apply(lambda x: 0 if (x is np.nan) else (2 if (x in (cattopnames)) else 1))
    modeldf['breeds.primary'] = modeldf['breeds.primary'].apply(lambda x: 1 if (x in cattopbreeds) else 0)

    # rename and reorder columns
    modeldf = modeldf.rename(columns={"name": "Name", "breeds.primary": "Breed", "colors.primary": "Color", "SexuponOutcome_Female-True": "SexuponOutcome_Spayed Female", 
    "SexuponOutcome_Female-False": "SexuponOutcome_Intact Female", "SexuponOutcome_Male-True": "SexuponOutcome_Neutered Male", "SexuponOutcome_Male-False": "SexuponOutcome_Intact Male",
    "age": "AgeuponOutcome", "day": "weekday"})

    modeldf = modeldf[['Name', 'AgeuponOutcome', 'Breed', 'Color', 'SexuponOutcome_Intact Female', 'SexuponOutcome_Intact Male', 'SexuponOutcome_Neutered Male', 
    'SexuponOutcome_Spayed Female', 'SexuponOutcome_Unknown', 'weekday', 'year', 'month']]
    #normalizing data
    MinMaxScaler = preprocessing.MinMaxScaler()
    X_data_minmax = MinMaxScaler.fit_transform(modeldf)
    modeldf = pd.DataFrame(X_data_minmax,columns=modeldf.columns)

    #print(modeldf.columns)
    loaded_model = pickle.load(open('cats_rf.pkl', 'rb'))
    query = modeldf
    query_pred = loaded_model.predict(query)
    print(df.columns)
    #new_df = df
    df['predicted_outcome'] = query_pred.tolist()
    #df['predicted_outcome'] = modeldf['predicted_outcome']
    #print(cats_json)
    return df

In [144]:
#cats_df = cats_df.drop('organization_id', axis=1)
df_new = getModelResults(cats_df)
df_new.head()

c:\Users\hadas\anaconda3\envs\musicrecs\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\hadas\anaconda3\envs\musicrecs\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description', 'organization_animal_id',
       'photos', 'videos', 'status', 'status_changed_at', 'published_at',
       'distance', 'breeds.primary', 'breeds.secondary', 'breeds.mixed',
       'breeds.unknown', 'colors.primary', 'colors.secondary',
       'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'primary_photo_cropped.small', 'primary_photo_cropped.medium',
       'primary_photo_cropped.large', 'primary_photo_cropped.full',
       'contact.email', 'contact.phone', 'contact.address.address1',
       'contact.address.address2', 'contact.address.city',
       'contact.address.state', 'contact.address.postcode',
       'contact.address.country', 'animal_id', 'animal_type',

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,animal_id,animal_type,organization_id,primary_photo_cropped,predicted_outcome
0,57323280,TX1393,https://www.petfinder.com/cat/clark-57323280/t...,Cat,Cat,Young,Male,Small,Short,"[Friendly, Affectionate, Gentle, Playful, Quie...",...,None,Houston,TX,77094,US,57323280,cat,tx1393,NaN,4
1,57323272,OH800,https://www.petfinder.com/cat/bernadette-57323...,Cat,Cat,Baby,Female,Medium,None,[],...,None,Wintersville,OH,43952,US,57323272,cat,oh800,NaN,0
2,57323273,OH800,https://www.petfinder.com/cat/terry-57323273/o...,Cat,Cat,Baby,Male,Medium,None,[],...,None,Wintersville,OH,43952,US,57323273,cat,oh800,NaN,0
3,57323265,TX170,https://www.petfinder.com/cat/stu-number-gru-a...,Cat,Cat,Young,Male,Small,None,[],...,None,Houston,TX,77005,US,57323265,cat,tx170,NaN,4
4,57323255,TX1660,https://www.petfinder.com/cat/adien-57323255/t...,Cat,Cat,Baby,Male,Medium,None,"[Friendly, Affectionate, Playful, Brave, Curio...",...,None,Justin,TX,76247,US,57323255,cat,tx1660,NaN,4


In [145]:
df_new.predicted_outcome.value_counts()

4    22
0    14
2    14
Name: predicted_outcome, dtype: int64

In [49]:
df = cats_df

ages = {'Baby': 0, 'Young': 1, 'Adult': 2, 'Senior': 3}
modeldf = df
modeldf["attributes.spayed_neutered"] = modeldf["attributes.spayed_neutered"].fillna("unknown")
modeldf['SexuponOutcome'] =  modeldf['gender'].astype(str) +"-"+ modeldf["attributes.spayed_neutered"].astype(str)
modeldf['year'] =  date.today().year
modeldf['month'] =  date.today().month
modeldf['day'] =  date.today().weekday()
modeldf['age'] = modeldf['age'].map(ages)
modeldf['age'] = modeldf['age'].fillna(2)
modeldf["colors.primary"] = modeldf["colors.primary"].fillna("unknown")
modeldf['SexuponOutcome_Unknown'] = modeldf['gender'].apply(lambda x: 0)
modeldf = modeldf[["breeds.primary", "SexuponOutcome", "SexuponOutcome_Unknown", "year", "month", "day", "age", "colors.primary", "name"]]
le = preprocessing.LabelEncoder()
modeldf['colors.primary'] = le.fit_transform(modeldf['colors.primary'])
modeldf = pd.get_dummies(data=modeldf, columns=['SexuponOutcome'])

modeldf['name'] = modeldf['name'].apply(lambda x: 0 if (x is np.nan) else (2 if (x in (cattopnames)) else 1))
modeldf['breeds.primary'] = modeldf['breeds.primary'].apply(lambda x: 1 if (x in cattopbreeds) else 0)

# rename and reorder columns
modeldf = modeldf.rename(columns={"name": "Name", "breeds.primary": "Breed", "colors.primary": "Color", "SexuponOutcome_Female-True": "SexuponOutcome_Spayed Female", 
"SexuponOutcome_Female-False": "SexuponOutcome_Intact Female", "SexuponOutcome_Male-True": "SexuponOutcome_Neutered Male", "SexuponOutcome_Male-False": "SexuponOutcome_Intact Male",
"age": "AgeuponOutcome", "day": "weekday"})

modeldf = modeldf[['Name', 'AgeuponOutcome', 'Breed', 'Color', 'SexuponOutcome_Intact Female', 'SexuponOutcome_Intact Male', 'SexuponOutcome_Neutered Male', 
'SexuponOutcome_Spayed Female', 'SexuponOutcome_Unknown', 'weekday', 'year', 'month']]
#normalizing data
MinMaxScaler = preprocessing.MinMaxScaler()
X_data_minmax = MinMaxScaler.fit_transform(modeldf)
modeldf = pd.DataFrame(X_data_minmax,columns=modeldf.columns)
loaded_model = pickle.load(open('cats_rf.pkl', 'rb'))
query = modeldf
query_pred = loaded_model.predict(query)
df["predicted_outcome"] = query_pred.tolist()

#query_pred
cats_json = df.to_json(orient='records')
#print(cats_json)

C:\Users\hadas\AppData\Local\Temp\ipykernel_3660\2687984975.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modeldf['colors.primary'] = le.fit_transform(modeldf['colors.primary'])
c:\Users\hadas\anaconda3\envs\musicrecs\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\hadas\anaconda3\envs\musicrecs\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.2 when

[{"id":57320868,"url":"https:\/\/www.petfinder.com\/cat\/felicia-57320868\/oh\/columbus\/buckeye-pet-partners-oh1210\/?referrer_id=4b67b1cd-0fc9-409d-bd57-d12b5bd0f185","type":"Cat","species":"Cat","age":2.0,"gender":"Female","size":"Medium","coat":"Short","tags":["Friendly","Gentle","Curious","loving","beautiful"],"name":"Felicia","description":"Felicia came to us as a pregnant teen-aged kitty.\nShe was too young to be a mother and sadly, only...","organization_animal_id":null,"photos":[{"small":"https:\/\/dl5zpyw5k3jeb.cloudfront.net\/photos\/pets\/57320868\/1\/?bust=1662920410&width=100","medium":"https:\/\/dl5zpyw5k3jeb.cloudfront.net\/photos\/pets\/57320868\/1\/?bust=1662920410&width=300","large":"https:\/\/dl5zpyw5k3jeb.cloudfront.net\/photos\/pets\/57320868\/1\/?bust=1662920410&width=600","full":"https:\/\/dl5zpyw5k3jeb.cloudfront.net\/photos\/pets\/57320868\/1\/?bust=1662920410"},{"small":"https:\/\/dl5zpyw5k3jeb.cloudfront.net\/photos\/pets\/57320868\/2\/?bust=1662920411&width

In [48]:
df

,id,url,type,species,age,gender,size,coat,tags,name,...,contact.address.country,animal_id,animal_type,primary_photo_cropped,SexuponOutcome,year,month,day,SexuponOutcome_Unknown,predicted_outcome
0,57320868,https://www.petfinder.com/cat/felicia-57320868...,Cat,Cat,2.0,Female,Medium,Short,"[Friendly, Gentle, Curious, loving, beautiful]",Felicia,...,US,57320868,cat,NaN,Female-True,2022,9,6,0,0
1,57321559,https://www.petfinder.com/cat/pavarotti-rhhs-s...,Cat,Cat,2.0,Male,Medium,None,[],Pavarotti (RHHS-Stray-rh214),...,US,57321559,cat,NaN,Male-False,2022,9,6,0,4
2,57321557,https://www.petfinder.com/cat/macheeba-40-doll...,Cat,Cat,2.0,Female,Small,None,[],MacHeeba - $40 Adoption Fee and FREE Gift Bag,...,US,57321557,cat,NaN,Female-True,2022,9,6,0,0
3,57321552,https://www.petfinder.com/cat/louie-and-george...,Cat,Cat,2.0,Male,Medium,Medium,"[Affectionate, Playful]",Louie and George,...,US,57321552,cat,NaN,Male-False,2022,9,6,0,4
4,57321556,https://www.petfinder.com/cat/herman-super-fri...,Cat,Cat,2.0,Male,Medium,Short,[],HERMAN - SUPER FRIENDLY TABBY GUY,...,US,57321556,cat,NaN,Male-True,2022,9,6,0,0
5,57321555,https://www.petfinder.com/cat/faye-57321555/ca...,Cat,Cat,2.0,Female,Medium,None,[],Faye,...,US,57321555,cat,NaN,Female-True,2022,9,6,0,0
6,57321545,https://www.petfinder.com/cat/hanson-57321545/...,Cat,Cat,2.0,Male,Medium,None,[],Hanson,...,US,57321545,cat,NaN,Male-True,2022,9,6,0,0
7,57321546,https://www.petfinder.com/cat/blinky-57321546/...,Cat,Cat,2.0,Female,Medium,None,[],Blinky,...,US,57321546,cat,NaN,Female-True,2022,9,6,0,0
8,57321538,https://www.petfinder.com/cat/rose-57321538/ok...,Cat,Cat,2.0,Female,Medium,Short,[],Rose,...,US,57321538,cat,NaN,Female-False,2022,9,6,0,4
9,57320559,https://www.petfinder.com/cat/marley-57320559/...,Cat,Cat,2.0,Female,Medium,Short,"[Friendly, Affectionate, Gentle, sweet, loving...",Marley,...,US,57320559,cat,NaN,Female-True,2022,9,6,0,0
